In [1]:
import pandas as pd

READING OF CSV

In [2]:
data=pd.read_csv('mobile_price_classification.csv')

In [3]:
data

,battery_power,bluetooth,clock_speed,dual_sim,front_cam,4G,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


TRAIN TEST SPLIT AND STANDARD SCALAR FOR NORMALIZATION

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [5]:

X = data.drop('price_range', axis=1)
y = data['price_range']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


SIMPLE ANN NUERAL NETWORK

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [7]:
# Define the model
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))  # 4 output classes for price range

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [8]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/50
40/40 [==============================] - 1s 4ms/step - loss: 1.3510 - accuracy: 0.3523 - val_loss: 1.2214 - val_accuracy: 0.5219
Epoch 2/50
40/40 [==============================] - 0s 2ms/step - loss: 1.1369 - accuracy: 0.5039 - val_loss: 0.9122 - val_accuracy: 0.6313
Epoch 3/50
40/40 [==============================] - 0s 2ms/step - loss: 0.8454 - accuracy: 0.6328 - val_loss: 0.6416 - val_accuracy: 0.7937
Epoch 4/50
40/40 [==============================] - 0s 2ms/step - loss: 0.6474 - accuracy: 0.7406 - val_loss: 0.4821 - val_accuracy: 0.8000
Epoch 5/50
40/40 [==============================] - 0s 2ms/step - loss: 0.5442 - accuracy: 0.7727 - val_loss: 0.3657 - val_accuracy: 0.9031
Epoch 6/50
40/40 [==============================] - 0s 2ms/step - loss: 0.4625 - accuracy: 0.8070 - val_loss: 0.3251 - val_accuracy: 0.8813
Epoch 7/50
40/40 [==============================] - 0s 2ms/step - loss: 0.4261 - accuracy: 0.8180 - val_loss: 0.2744 - val_accuracy: 0.9094
Epoch 8/50
40/40 [==

In [9]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test accuracy: {test_accuracy:.4f}')


Test accuracy: 0.9250


In [10]:
!pip install keras-tuner


Defaulting to user installation because normal site-packages is not writeable


HYPERPARAMETER TUNING USING RANDOM SEARCH 

In [11]:
from kerastuner import RandomSearch

def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units_1', min_value=32, max_value=512, step=32), input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(hp.Int('units_2', min_value=32, max_value=256, step=32), activation='relu'))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(4, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Use RandomSearch to find the best hyperparameters
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=10, executions_per_trial=1, directory='tuner', project_name='mobile_price')

tuner.search(X_train, y_train, epochs=50, validation_split=0.2)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f'Test accuracy after hyperparameter tuning: {test_accuracy:.4f}')


Reloading Tuner from tuner/mobile_price/tuner0.json


<ipython-input-11-bf8da014627d>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


Test accuracy after hyperparameter tuning: 0.9250
